# Text Embeddings Generation

**Objectif**: Générer embeddings textuels (CamemBERT 768 dim)

**Prérequis**: Données intégrées avec classification IID/OOD, textes combinés (Nom + Definition) --> data_integration_annotation.ipynb

In [10]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

DATA_DIR = '../data/'
MODEL = 'dangvantuan/sentence-camembert-base'
print(" Configuration chargée")

 Configuration chargée


## 1. Charger données + préparer textes
'Nom' + 'Definition' combinés en 'text' pour chaque produit. Vérification du nombre de produits, distribution IID/OOD, nombre de textes uniques.

In [11]:
# Charger données
df = pd.read_excel(DATA_DIR + 'data_merged_with_ood_classification.xlsx')
print(f" Chargé: {len(df)} produits")
print(f"  - IID: {(df['OOD']==0).sum()}, OOD: {(df['OOD']==1).sum()}")

# Textes combinés
df['text'] = df['Nom'].astype(str) + "; " + df['Definition'].astype(str)
unique_texts = df['text'].drop_duplicates().tolist()

print(f"\n Textes: {len(df)} produits, {len(unique_texts)} uniques")

 Chargé: 6550 produits
  - IID: 6428, OOD: 122

 Textes: 6550 produits, 261 uniques


## 2. Générer embeddings (CamemBERT 768 dim)

In [12]:
# Charger modèle
print(f"Chargement {MODEL}...")
model = SentenceTransformer(MODEL)

# Embeddings textes uniques
print(f"Génération embeddings {len(unique_texts)} textes...")
emb_unique = model.encode(unique_texts, show_progress_bar=True, batch_size=32)

# Mapping pour tous les produits
text_to_idx = {text: idx for idx, text in enumerate(unique_texts)}
indices = df['text'].map(text_to_idx).values
emb_all = emb_unique[indices]

print(f"\n Embeddings générés")
print(f"  - Unique: {emb_unique.shape}")
print(f"  - Tous: {emb_all.shape}")

# Sauvegarder
np.save(DATA_DIR + 'embeddings_unique.npy', emb_unique)
np.save(DATA_DIR + 'embeddings_combined.npy', emb_all)
print(f" Sauvegardés")

Chargement dangvantuan/sentence-camembert-base...
Génération embeddings 261 textes...


Batches:   0%|          | 0/9 [00:00<?, ?it/s]


 Embeddings générés
  - Unique: (261, 768)
  - Tous: (6550, 768)
 Sauvegardés


## 3. Export final
Ajout des embeddings au dataframe, sauvegarde en .npy pour les embeddings, .xlsx pour le dataframe enrichi.

In [13]:
# Ajouter indices et sauvegarder
df['embedding_idx'] = indices
df.to_excel(DATA_DIR + 'data_with_embeddings_index.xlsx', index=False)

print(f" Export finalisé")
print(f"\n Résumé:")
print(f"  - Produits: {len(df)}")
print(f"  - Embeddings 768D: embeddings_combined.npy ({emb_all.nbytes/1e6:.1f} MB)")
print(f"  - Embeddings unique: embeddings_unique.npy ({emb_unique.nbytes/1e6:.1f} MB)")
print(f"  - Données: data_with_embeddings_index.xlsx")

 Export finalisé

 Résumé:
  - Produits: 6550
  - Embeddings 768D: embeddings_combined.npy (20.1 MB)
  - Embeddings unique: embeddings_unique.npy (0.8 MB)
  - Données: data_with_embeddings_index.xlsx


## Visualisation 3D PCA

In [14]:
import sys
sys.path.insert(0, '../')

# Charger embeddings et données
embeddings = np.load(DATA_DIR + 'embeddings_combined.npy')
df_with_embeddings = pd.read_excel(DATA_DIR + 'data_with_embeddings_index.xlsx')

from src.visu import plot_embeddings_3d_pca

# Visualisation par OOD (défaut: bleu=IID, rouge=OOD)
fig_ood, pca = plot_embeddings_3d_pca(
    embeddings, 
    df_with_embeddings, 
    hue_col='OOD',
    title='PCA 3D - Embeddings CamemBERT (par OOD)'
)
fig_ood.show()

print(f"\n✓ Visualisation PCA 3D par OOD")
print(f"  - IID (Bleu): {(df_with_embeddings['OOD']==0).sum()} échantillons")
print(f"  - OOD (Rouge): {(df_with_embeddings['OOD']==1).sum()} échantillons")
print(f"  - Variance expliquée (PC1+PC2+PC3): {pca.explained_variance_ratio_[:3].sum():.1%}")

# Tu peux aussi visualiser par d'autres colonnes:
# fig_classe, _ = plot_embeddings_3d_pca(embeddings, df_with_embeddings, hue_col='Classe', title='PCA 3D - par Classe')
fig_nom, _ = plot_embeddings_3d_pca(embeddings, df_with_embeddings, hue_col='Nom', title='PCA 3D - par Nom')
fig_nom.show()


✓ Visualisation PCA 3D par OOD
  - IID (Bleu): 6428 échantillons
  - OOD (Rouge): 122 échantillons
  - Variance expliquée (PC1+PC2+PC3): 27.5%
